In [1]:
import os
import mysql.connector

In [2]:
import subprocess
import json  # Importing the json module
import pandas as pd
from datetime import datetime


#AGGREGATOR TRANSACTION 
############################################################ 
path1 = "C:/AL ML/Project2-phonepe/pulse/data/aggregated/transaction/country/india/state/"
agg_trans_list = os.listdir(path1)
#print(agg_trans_list)

columns1 = {"States":[],"Year": [],"Quater":[],"TransactionType": [], "TransactionCount":[],"TransactionAmount":[]}

# setting the path of each state
for state in agg_trans_list:
    curr_states = path1+state+"/"
    #print(curr_states)
    year_list = os.listdir(curr_states)
    #print(year_list)
    for yr in year_list:
        curr_yr = curr_states+yr+"/"
        #print(curr_yr)
        yearFileList  = os.listdir(curr_yr)
        #print(yearFileList)
        for file in yearFileList:
            data = open(curr_yr+file,"r")
            #print(data)

            A = json.load(data)
            for i in A["data"]["transactionData"]:
              name = i["name"]
              count = i["paymentInstruments"][0]["count"]
              amount = i["paymentInstruments"][0]["amount"]

              columns1["TransactionType"].append(name)
              columns1["TransactionCount"].append(count)
              columns1["States"].append(state)
              columns1["Year"].append(yr)
              columns1["Quater"].append(int(file.split(".")[0]))
              columns1["TransactionAmount"].append(amount)
#print(columns1)
aggregator_transaction  = pd.DataFrame(columns1)
#print(aggregator_transaction)

connection = mysql.connector.connect(
    host="localhost",          # Your MySQL server address
    port= "3306",
    user="root",      # Your MySQL username
    password="Sai@12345",  # Your MySQL password
    database="PhonePeDB"   # Your MySQL database name
)
cursor = connection.cursor()

# Delete the table if it exists
delete_table_query = "DROP TABLE IF EXISTS agg_transactions;"
cursor.execute(delete_table_query)

# Create a new table

create_query ="""CREATE TABLE IF NOT EXISTS agg_transactions (
    id INT AUTO_INCREMENT PRIMARY KEY,
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    TransactionType VARCHAR(255),
    TransactionCount INT,
    TransactionAmount FLOAT
);"""

cursor.execute(create_query)

for index, row in aggregator_transaction.iterrows():
    insert_query = """
    INSERT INTO agg_transactions (State, Year, Quarter, TransactionType, TransactionCount, TransactionAmount)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    data_tuple = (
        row["States"],  # Adjusted to "States" to match DataFrame column names
        row["Year"],
        int(row["Quater"]),  # Adjusted to "Quater" to match DataFrame column names
        row["TransactionType"],
        row["TransactionCount"],
        row["TransactionAmount"]
    )
    cursor.execute(insert_query, data_tuple)

# Commit the transaction
connection.commit()


#AGGREGATOR INSURANCE
##############################################################################

# Define the path for the data
path2 = "C:/AL ML/Project2-phonepe/pulse/data/aggregated/insurance/country/india/state/"
agg_insurance_list = os.listdir(path2)

# Initialize the columns for the DataFrame
columns2 = {
    "States": [],
    "Year": [],
    "Quater": [],
    "TransactionType": [],
    "TransactionCount": [],
    "TransactionAmount": [],
    "From": [],
    "To": []
}

# Parse through each state and year to extract the data
for state in agg_insurance_list:
    curr_states = path2 + state + "/"
    year_list = os.listdir(curr_states)
    for yr in year_list:
        curr_yr = curr_states + yr + "/"
        yearFileList = os.listdir(curr_yr)
        #print(yearFileList)
        for file in yearFileList:
            with open(curr_yr + file, "r") as data:
                B = json.load(data)
                from_timestamp = B["data"]["from"]
                to_timestamp = B["data"]["to"]

                # Convert milliseconds to datetime objects
                from_date = datetime.utcfromtimestamp(from_timestamp / 1000)
                to_date = datetime.utcfromtimestamp(to_timestamp / 1000)

                for i in B["data"]["transactionData"]:
                    name = i["name"]
                    count = i["paymentInstruments"][0]["count"]
                    amount = i["paymentInstruments"][0]["amount"]

                    columns2["TransactionType"].append(name)
                    columns2["TransactionCount"].append(count)
                    columns2["States"].append(state)
                    columns2["Year"].append(yr)
                    columns2["Quater"].append(int(file.split(".")[0]))
                    columns2["TransactionAmount"].append(amount)
                    columns2["From"].append(from_date)
                    columns2["To"].append(to_date)

# Create a DataFrame with the parsed data
aggregator_insurance = pd.DataFrame(columns2)
#print(aggregator_insurance)



# Drop the table if it exists
delete_table_query = "DROP TABLE IF EXISTS agg_insurance;"
cursor.execute(delete_table_query)

# Create the new table with the necessary columns
create_table_query = """
CREATE TABLE agg_insurance (
    id INT AUTO_INCREMENT PRIMARY KEY,
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    TransactionType VARCHAR(255),
    TransactionCount INT,
    TransactionAmount FLOAT,
    `From` DATETIME,
    `To` DATETIME
);
"""
cursor.execute(create_table_query)  # Corrected line

# Insert the data into the newly created table
for index, row in aggregator_insurance.iterrows():
    insert_query = """
    INSERT INTO agg_insurance (State, Year, Quarter, TransactionType, TransactionCount, TransactionAmount, `From`, `To`)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    data_tuple = (
        row["States"],
        row["Year"],
        int(row["Quater"]),
        row["TransactionType"],
        row["TransactionCount"],
        row["TransactionAmount"],
        row["From"],
        row["To"]
    )
    cursor.execute(insert_query, data_tuple)

# Commit the transaction
connection.commit()


#AGGREGATOR USER
######################################################################################## 

path3 = "C:/AL ML/Project2-phonepe/pulse/data/aggregated/user/country/india/state/"
agg_user_list = os.listdir(path3)
#print(agg_trans_list)

columns3 = {"Brands":[],"Year": [],"Quater":[], "TransactionCount":[],"Percentage":[],"States":[]}

# setting the path of each state
for state in agg_user_list:
    curr_states = path3+state+"/"
    #print(curr_states)
    year_list = os.listdir(curr_states)
    #print(year_list)
    for yr in year_list:
        curr_yr = curr_states+yr+"/"
        #print(curr_yr)
        yearFileList  = os.listdir(curr_yr)
        #print(yearFileList)
        for file in yearFileList:
            data = open(curr_yr+file,"r")
        
            C = json.load(data)
           
            try:
                for i in  C["data"]["usersByDevice"]:
                    brand = i["brand"]
                    count = i["count"]
                    percentage = i["percentage"]

                    columns3["Brands"].append(brand)
                    columns3["TransactionCount"].append(count)
                    columns3["Percentage"].append(percentage)
                    columns3["Year"].append(yr)
                    columns3["Quater"].append(int(file.split(".")[0]))
                    columns3["States"].append(state)
            except:
                pass

aggregator_user = pd.DataFrame(columns3)

delete_table_query = "DROP TABLE IF EXISTS agg_user;"
cursor.execute(delete_table_query)

# Create the new table with the necessary columns
create_table_query = """
CREATE TABLE agg_user (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Brands VARCHAR(255),
    Year INT,
    Quarter INT,
    TransactionCount INT,
    Percentage FLOAT,
    State VARCHAR(255)
);
"""
cursor.execute(create_table_query)

# Insert the data into the newly created table
for index, row in aggregator_user.iterrows():
    insert_query = """
    INSERT INTO agg_user (Brands, Year, Quarter, TransactionCount, Percentage, State)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    data_tuple = (
        row["Brands"],
        row["Year"],
        int(row["Quater"]),
        row["TransactionCount"],
        row["Percentage"],
        row["States"]
    )
    cursor.execute(insert_query, data_tuple)

# Commit the transaction
connection.commit()



In [54]:
#MAP transaction 
#####################################################################################C:\AL ML\Project2-phonepe\pulse\data\map\transaction
path4 = "C:/AL ML/Project2-phonepe/pulse/data/map/transaction/hover/country/india/state/"
map_trans_list = os.listdir(path4)
#print(map_trans_list)

columns4 = {"States":[],"Year": [],"Quater":[],"TransactionType": [], "TransactionCount":[],"TransactionAmount":[]}

# setting the path of each state
for state in map_trans_list:
    curr_states = path4+state+"/"
    #print(curr_states)
    year_list = os.listdir(curr_states)
    #print(year_list)
    for yr in year_list:
        curr_yr = curr_states+yr+"/"
        #print(curr_yr)
        yearFileList  = os.listdir(curr_yr)
        print(yearFileList)
        for file in yearFileList:
            data = open(curr_yr+file,"r")
            #print(data)
            D= json.load(data)
            for item in D["data"]["hoverDataList"]:
                    name = item["name"]
                    for metric in item["metric"]:
                        transaction_type = metric["type"]
                        count = metric["count"]
                        amount = metric["amount"]
                        
                        # Append the extracted data to the columns4 dictionary
                        columns4["TransactionType"].append(transaction_type)
                        columns4["TransactionCount"].append(count)
                        columns4["TransactionAmount"].append(amount)
                        columns4["States"].append(state)
                        columns4["Year"].append(yr)
                        columns4["Quater"].append(int(file.split(".")[0]))

# Convert the dictionary to a DataFrame
map_transaction = pd.DataFrame(columns4)
#print(map_transaction)



# Delete the table if it exists
delete_table_query = "DROP TABLE IF EXISTS map_transactions;"
cursor.execute(delete_table_query)

# Create a new table
create_query ="""CREATE TABLE IF NOT EXISTS map_transactions (
    id INT AUTO_INCREMENT PRIMARY KEY,
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    TransactionType VARCHAR(255),
    TransactionCount INT,
    TransactionAmount FLOAT
);"""

cursor.execute(create_query)

for index, row in map_transaction.iterrows():
    insert_query = """
    INSERT INTO map_transactions (State, Year, Quarter, TransactionType, TransactionCount, TransactionAmount)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    data_tuple = (
        row["States"],  # Adjusted to "States" to match DataFrame column names
        row["Year"],
        int(row["Quater"]),  # Adjusted to "Quater" to match DataFrame column names
        row["TransactionType"],
        row["TransactionCount"],
        row["TransactionAmount"]
    )
    cursor.execute(insert_query, data_tuple)

# Commit the transaction
connection.commit()


#MAP insurance 
###########################################################################################

# Define the path for the data
path5 = "C:/AL ML/Project2-phonepe/pulse/data/map/insurance/country/india/state/"
map_insurance_list = os.listdir(path5)

# Initialize the columns for the DataFrame
columns5 = {
    "States": [],
    "Year": [],
    "Quater": [],
    "District": [],
    "Latitude": [],
    "Longitude": [],
    "Metric": [],
}

# Parse through each state and year to extract the data
for state in map_insurance_list:
    curr_states = path5 + state + "/"
    year_list = os.listdir(curr_states)
    for yr in year_list:
        curr_yr = curr_states + yr + "/"
        yearFileList = os.listdir(curr_yr)
        for file in yearFileList:
            with open(curr_yr + file, "r") as data:
                E = json.load(data)
                for entry in E["data"]["data"]["data"]:
                    latitude = entry[0]  # Latitude
                    longitude = entry[1]  # Longitude
                    metric = entry[2]  # Metric
                    district = entry[3]  # District label
                    print("latitude")
                    print(latitude)
                    # Append the data to the DataFrame columns
                    columns5["States"].append(state)
                    columns5["Year"].append(yr)
                    columns5["Quater"].append(int(file.split(".")[0]))
                    columns5["District"].append(district)
                    columns5["Latitude"].append(latitude)
                    columns5["Longitude"].append(longitude)
                    columns5["Metric"].append(metric)

# Create a DataFrame with the parsed data
map_insurance = pd.DataFrame(columns5)

# Drop the table if it exists
delete_table_query = "DROP TABLE IF EXISTS map_insurance;"
cursor.execute(delete_table_query)

# Create the new table with the necessary columns
create_table_query = """
CREATE TABLE map_insurance (
    id INT AUTO_INCREMENT PRIMARY KEY,
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    District VARCHAR(255),
    Latitude DECIMAL(10, 8),
    Longitude DOUBLE,
    Metric FLOAT
);
"""
cursor.execute(create_table_query)

# Insert the data into the newly created table
for index, row in map_insurance.iterrows():
    insert_query = """
    INSERT INTO map_insurance (State, Year, Quarter, District, Latitude, Longitude, Metric)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    data_tuple = (
        row["States"],
        row["Year"],
        row["Quater"],
        row["District"],
        row["Latitude"],
        row["Longitude"],
        row["Metric"]
    )
    cursor.execute(insert_query, data_tuple)

# Commit the transaction
connection.commit()


#MAP user 
#############################################################################################

path6 = "C:/AL ML/Project2-phonepe/pulse/data/map/user/hover/country/india/state/"
map_user_list = os.listdir(path6)
#print(map_user_list)

columns6 = {"Brands":[],"Year": [],"Quater":[], "TransactionCount":[],"Percentage":[],"States":[]}

# setting the path of each state
for state in map_user_list:
    curr_states = path3+state+"/"
    #print(curr_states)
    year_list = os.listdir(curr_states)
    #print(year_list)
    for yr in year_list:
        curr_yr = curr_states+yr+"/"
        #print(curr_yr)
        yearFileList  = os.listdir(curr_yr)
        #print(yearFileList)
        for file in yearFileList:
            data = open(curr_yr+file,"r")
        
            F = json.load(data)
           
            try:
                for i in  F["data"]["usersByDevice"]:
                    brand = i["brand"]
                    count = i["count"]
                    percentage = i["percentage"]

                    columns3["Brands"].append(brand)
                    columns3["TransactionCount"].append(count)
                    columns3["Percentage"].append(percentage)
                    columns3["Year"].append(yr)
                    columns3["Quater"].append(int(file.split(".")[0]))
                    columns3["States"].append(state)
            except:
                pass

map_user = pd.DataFrame(columns3)

delete_table_query = "DROP TABLE IF EXISTS map_user;"
cursor.execute(delete_table_query)

# Create the new table with the necessary columns
create_table_query = """
CREATE TABLE map_user (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Brands VARCHAR(255),
    Year INT,
    Quarter INT,
    TransactionCount INT,
    Percentage FLOAT,
    State VARCHAR(255)
);
"""
cursor.execute(create_table_query)

# Insert the data into the newly created table
for index, row in map_user.iterrows():
    insert_query = """
    INSERT INTO map_user (Brands, Year, Quarter, TransactionCount, Percentage, State)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    data_tuple = (
        row["Brands"],
        row["Year"],
        int(row["Quater"]),
        row["TransactionCount"],
        row["Percentage"],
        row["States"]
    )
    cursor.execute(insert_query, data_tuple)

# Commit the transaction
connection.commit()



['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.json', '4.json']
['1.json', '2.json', '3.

In [3]:
#Top INSURANCE
#########################################################################################

path7 = "C:/AL ML/Project2-phonepe/pulse/data/top/insurance/country/india/state/"
top_insurance_list = os.listdir(path7)

# Initialize the columns for the DataFrame
columns7 = {
    "States": [],
    "Year": [],
    "Quater": [],
    "TransactionAmount": [],
    "TransactionCount": [],
    "Pincode": []  
}

# Parse through each state and year to extract the data
for state in top_insurance_list:
    curr_states = path7 + state + "/"
    year_list = os.listdir(curr_states)
    for yr in year_list:
        curr_yr = curr_states + yr + "/"
        yearFileList = os.listdir(curr_yr)
        for file in yearFileList:
            with open(curr_yr + file, "r") as data:
                G = json.load(data)
                for pin in G["data"]["pincodes"]:
                    entityname = pin["entityName"]
                    count = pin["metric"]["count"]
                    amount = pin["metric"]["amount"]
                    columns7["Year"].append(yr)
                    columns7["Quater"].append(int(file.split(".")[0]))
                    columns7["States"].append(state)
                    columns7["TransactionAmount"].append(amount)
                    columns7["TransactionCount"].append(count)
                    columns7["Pincode"].append(entityname)
top_insurance = pd.DataFrame(columns7)



# Drop the table if it exists
delete_table_query = "DROP TABLE IF EXISTS top_insurance;"
cursor.execute(delete_table_query)

# Create the new table with the necessary columns
create_table_query = """
CREATE TABLE top_insurance (
    id INT AUTO_INCREMENT PRIMARY KEY,
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    TransactionAmount FLOAT,
    TransactionCount INT,
    Pincode VARCHAR(20)
);
"""
cursor.execute(create_table_query)

# Insert the data into the newly created table
for index, row in top_insurance.iterrows():
    insert_query = """
    INSERT INTO top_insurance (State, Year, Quarter, TransactionAmount, TransactionCount, Pincode)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    data_tuple = (
        row["States"],
        row["Year"],
        int(row["Quater"]),
        row["TransactionAmount"],
        row["TransactionCount"],
        row["Pincode"]
    )
    cursor.execute(insert_query, data_tuple)


path_transaction = "C:/AL ML/Project2-phonepe/pulse/data/top/transaction/country/india/state/"
top_transaction_list = os.listdir(path_transaction)

# Initialize the columns for the DataFrame
columns_transaction = {
    "States": [],
    "Year": [],
    "Quater": [],
    "TransactionAmount": [],
    "TransactionCount": [],
    "Pincode": []  
}

# Parse through each state and year to extract the data
for state in top_transaction_list:
    curr_states = path_transaction + state + "/"
    year_list = os.listdir(curr_states)
    for yr in year_list:
        curr_yr = curr_states + yr + "/"
        yearFileList = os.listdir(curr_yr)
        for file in yearFileList:
            with open(curr_yr + file, "r") as data:
                G = json.load(data)
                for pin in G["data"]["pincodes"]:
                    entityname = pin["entityName"]
                    count = pin["metric"]["count"]
                    amount = pin["metric"]["amount"]
                    columns_transaction["Year"].append(yr)
                    columns_transaction["Quater"].append(int(file.split(".")[0]))
                    columns_transaction["States"].append(state)
                    columns_transaction["TransactionAmount"].append(amount)
                    columns_transaction["TransactionCount"].append(count)
                    columns_transaction["Pincode"].append(entityname)
top_transaction = pd.DataFrame(columns_transaction)

# Drop the existing table
delete_table_query_transaction = "DROP TABLE IF EXISTS top_transaction;"
cursor.execute(delete_table_query_transaction)

# Create the new table
create_table_query_transaction = """
CREATE TABLE top_transaction (
    id INT AUTO_INCREMENT PRIMARY KEY,
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    TransactionAmount FLOAT,
    TransactionCount INT,
    Pincode VARCHAR(20)
);
"""
cursor.execute(create_table_query_transaction)

# Insert data into the table
for index, row in top_transaction.iterrows():
    insert_query_transaction = """
    INSERT INTO top_transaction (State, Year, Quarter, TransactionAmount, TransactionCount, Pincode)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    data_tuple_transaction = (
        row["States"],
        row["Year"],
        int(row["Quater"]),
        row["TransactionAmount"],
        row["TransactionCount"],
        row["Pincode"]
    )
    cursor.execute(insert_query_transaction, data_tuple_transaction)

path_user = "C:/AL ML/Project2-phonepe/pulse/data/top/user/country/india/state/"
top_user_list = os.listdir(path_user)

# Initialize the columns for the DataFrame
columns_user = {
    "States": [],
    "Year": [],
    "Quater": [],
    "RegisteredUsers": [],
    "Pincode": []  
}

# Parse through each state and year to extract the data
for state in top_user_list:
    curr_states = path_user + state + "/"
    year_list = os.listdir(curr_states)
    for yr in year_list:
        curr_yr = curr_states + yr + "/"
        yearFileList = os.listdir(curr_yr)
        for file in yearFileList:
            with open(curr_yr + file, "r") as data:
                G = json.load(data)
                for pin in G["data"]["pincodes"]:
                    entityname = pin["entityName"]
                    registered_users = pin["metric"]["registeredUsers"]
                    columns_user["Year"].append(yr)
                    columns_user["Quater"].append(int(file.split(".")[0]))
                    columns_user["States"].append(state)
                    columns_user["RegisteredUsers"].append(registered_users)
                    columns_user["Pincode"].append(entityname)
top_user = pd.DataFrame(columns_user)

# Drop the existing table
delete_table_query_user = "DROP TABLE IF EXISTS top_user;"
cursor.execute(delete_table_query_user)

# Create the new table
create_table_query_user = """
CREATE TABLE top_user (
    id INT AUTO_INCREMENT PRIMARY KEY,
    State VARCHAR(255),
    Year INT,
    Quarter INT,
    RegisteredUsers INT,
    Pincode VARCHAR(20)
);
"""
cursor.execute(create_table_query_user)

# Insert data into the table
for index, row in top_user.iterrows():
    insert_query_user = """
    INSERT INTO top_user (State, Year, Quarter, RegisteredUsers, Pincode)
    VALUES (%s, %s, %s, %s, %s)
    """
    data_tuple_user = (
        row["States"],
        row["Year"],
        int(row["Quater"]),
        row["RegisteredUsers"],
        row["Pincode"]
    )
    cursor.execute(insert_query_user, data_tuple_user)


# Commit the transaction
connection.commit()


: 